In [ ]:
import os
import numpy as np
import jax
import jax.numpy as jnp
import arviz as az

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS

/home/jiapeng/miniconda3/envs/drt_work/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


functions to generate synthetic impedance and corresponding DRT profiles

In [2]:
def fct_Z_HN_exact(freq_vec, R_inf, R_ct, phi, zeta, tau_0):
    """
    Computes exact impedance for an HN model:
        Z(omega) = R_inf + R_ct / [1 + (j * omega * tau_0)^phi]^alpha.

    Returns both:
      - Z_exact:       complex array of shape (N_freqs,)
      - Z_exact_re_im: real array of shape (2*N_freqs,) storing real & imag parts
    """
    omega = 2.0 * np.pi * freq_vec
    Z_exact = R_inf + R_ct / ((1.0 + (1j * omega * tau_0)**phi) ** zeta)

    n_freqs = len(freq_vec)
    Z_exact_re_im = np.zeros(2 * n_freqs)
    Z_exact_re_im[:n_freqs] = Z_exact.real
    Z_exact_re_im[n_freqs:] = Z_exact.imag

    return Z_exact, Z_exact_re_im

def fct_Z_synth(Z_exact, sigma_n_exp=0.2, random_seed=1225):
    """
    Adds synthetic Gaussian noise to the real and imaginary parts of the exact
    impedance, simulating experimental data.

    Args:
      Z_exact:     complex array, shape (N_freqs,)
      sigma_n_exp: standard deviation of added noise
      random_seed: seed for reproducibility

    Returns:
      Z_exp:       noisy (synthetic) complex array, shape (N_freqs,)
      Z_exp_re_im: real array, shape (2*N_freqs,) with Re and Im parts.
    """
    N_freqs = len(Z_exact)
    np.random.seed(random_seed)
    noise_real = np.random.normal(0, sigma_n_exp, N_freqs)
    noise_imag = np.random.normal(0, sigma_n_exp, N_freqs)
    Z_exp = Z_exact + (noise_real + 1j*noise_imag)

    Z_exp_re_im = np.zeros(2*N_freqs)
    Z_exp_re_im[:N_freqs] = Z_exp.real
    Z_exp_re_im[N_freqs:] = Z_exp.imag
    
    return Z_exp, Z_exp_re_im

def theta_HN(tau_vec, tau0, phi):
    """
    Helper function to compute the argument theta_HN(tau).
    """
    return np.arctan2(
        np.sin(np.pi * phi),
        ((tau_vec / tau0)**phi + np.cos(np.pi * phi))
    )

def gamma_HN(tau_vec, w, phi, zeta, tau0):
    """
    Distribution of Relaxation Times for the HN model.
    gamma_HN(tau) = (w / pi) * (tau/tau0)^(zeta*phi) * sin(zeta*theta)
                    / [1 + 2*cos(pi*phi)*(tau/tau0)^phi + (tau/tau0)^(2phi)]^(zeta/2)
    Interprets 'w' as the arc amplitude (analogous to R_ct).
    """
    ratio_phi = (tau_vec / tau0)**phi
    prefactor = w / np.pi
    th = theta_HN(tau_vec, tau0, phi)

    numerator = (tau_vec / tau0)**(zeta * phi) * np.sin(zeta * th)
    denom = (1.0 + 2.0*np.cos(np.pi * phi)*ratio_phi + ratio_phi**2)**(zeta / 2.0)

    return prefactor * (numerator / denom)

In [3]:
def HN_element(freq_array, w, phi, zeta, log_tau0):
    """
    Single 'HN' element:
      Z_k = w / [1 + (j * 2πf * tau0)^phi]^zeta
    """
    tau0 = jnp.exp(log_tau0)
    omega = 2.0 * jnp.pi * freq_array
    return w / ((1.0 + (1j * omega * tau0)**phi) ** zeta)

def stick_breaking(beta):
    """
    Produce pi_1,...,pi_K from Beta(1, alpha) draws (truncated DP).
    """
    pi_list = []
    prod = 1.0
    for b in beta:
        portion = b * prod
        pi_list.append(portion)
        prod *= (1.0 - b)
    pi_list.append(prod)
    return jnp.stack(pi_list)

In [4]:
def dp_HN_model(freq_data, Z_re_data, Z_im_data, K=10):
    """
    Truncated DP-HN model for EIS data.
    """
    # Global R_inf
    R_inf = numpyro.sample("R_inf", dist.HalfNormal(50.0))

    # DP concentration alpha
    alpha_dp = numpyro.sample("alpha_dp", dist.Exponential(1.0))

    # Stick-breaking to get mixing coefficients
    beta_vals = numpyro.sample(
        "beta_vals", dist.Beta(jnp.ones(K - 1), alpha_dp * jnp.ones(K - 1))
    )
    pi_vals = stick_breaking(beta_vals)  # shape (K,)

    # Base distribution for each arc
    w_vals   = numpyro.sample("w_vals",   dist.HalfNormal(50.0).expand([K]))
    phi_vals = numpyro.sample("phi_vals", dist.Uniform(0.0, 1.0).expand([K]))
    alpha_vals = numpyro.sample("alpha_vals", dist.Uniform(0.0, 1.0).expand([K]))
    log_tau_vals = numpyro.sample("log_tau_vals", dist.Normal(0.0, 2.0).expand([K]))

    # Noise
    sigma_re = numpyro.sample("sigma_re", dist.HalfNormal(0.3))
    sigma_im = numpyro.sample("sigma_im", dist.HalfNormal(0.3))

    def single_freq_impedance(f):
        Z_sum = R_inf + 0.0j
        for k in range(K):
            Z_k = HN_element(f, w_vals[k], phi_vals[k], alpha_vals[k], log_tau_vals[k])
            Z_sum += pi_vals[k] * Z_k
        return Z_sum

    Z_pred = jax.vmap(single_freq_impedance)(freq_data)
    Z_pred_re = Z_pred.real
    Z_pred_im = Z_pred.imag

    # Likelihood
    numpyro.sample("obs_re", dist.Normal(Z_pred_re, sigma_re), obs=Z_re_data)
    numpyro.sample("obs_im", dist.Normal(Z_pred_im, sigma_im), obs=Z_im_data)

1) Generates a 2-arc synthetic impedance data set (HN model) and its exact DRT.

In [5]:
# 1) Generate synthetic data (HN-based)
N_freqs = 81                                # Frequency range
freq_min, freq_max = 1e-2, 1e6
freq_vec = np.logspace(np.log10(freq_min), np.log10(freq_max), num=N_freqs)

# Arc #1
R_inf_1, R_ct_1, phi_1, zeta_1, tau_1 = 10.0, 50.0, 0.8, 0.8, 1e-1
# Arc #2
R_inf_2, R_ct_2, phi_2, zeta_2, tau_2 = 0.0, 20.0, 0.8, 0.8, 1e-4

Z_exact_1, _ = fct_Z_HN_exact(freq_vec, R_inf_1, R_ct_1, phi_1, zeta_1, tau_1)
Z_exact_2, _ = fct_Z_HN_exact(freq_vec, R_inf_2, R_ct_2, phi_2, zeta_2, tau_2)
Z_exact = Z_exact_1 + Z_exact_2

# Add Gaussian noise
Z_exp, _ = fct_Z_synth(Z_exact, sigma_n_exp=0.2, random_seed=1225)

# DRT reference (sum of both arcs)
N_tau = 801
tau_vec = np.logspace(-np.log10(freq_max), -np.log10(freq_min), num=N_tau)

gamma_exact_1 = gamma_HN(tau_vec, R_ct_1, phi_1, zeta_1, tau_1)
gamma_exact_2 = gamma_HN(tau_vec, R_ct_2, phi_2, zeta_2, tau_2)
gamma_exact = gamma_exact_1 + gamma_exact_2

2) Run truncated DP-HN with K arcs

In [ ]:
K = 10
nuts_kernel = NUTS(dp_HN_model)
mcmc = MCMC(nuts_kernel, num_warmup=80, num_samples=150, num_chains=3)
mcmc.run(
    jax.random.PRNGKey(0),
    freq_data=jnp.array(freq_vec),
    Z_re_data=jnp.array(Z_exp.real),
    Z_im_data=jnp.array(Z_exp.imag),
    K=K
)

posterior_samples = mcmc.get_samples()
az_data = az.from_numpyro(mcmc)

3) Save all data into 'dp_hn_results' instead of 'dp_zarc_results'

In [ ]:
# print("\n===== Posterior Summary =====")
# # Note: 'alpha_vals' replaces the old 'phi_vals' for the shape parameter alpha
# print(az.summary(az_data, var_names=[
#     "R_inf","alpha_dp","w_vals","phi_vals","alpha_vals","log_tau_vals","sigma_re","sigma_im"
# ]))

# os.makedirs("dp_hn_results", exist_ok=True)
# # Posterior samples
# # Convert JAX arrays to NumPy arrays
# np.save("dp_hn_results/posterior_samples.npy",
#         {k: np.array(v) for k, v in posterior_samples.items()})

# # Generated data, references
# np.save("dp_hn_results/freq_vec.npy",    freq_vec)
# np.save("dp_hn_results/Z_exp.npy",       Z_exp)
# np.save("dp_hn_results/Z_exact.npy",     Z_exact)
# np.save("dp_hn_results/tau_vec.npy",     tau_vec)
# np.save("dp_hn_results/gamma_exact.npy", gamma_exact)

# print("\nSampling complete. Results saved to dp_hn_results/*.npy")